In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
import cufflinks as cf
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
cf.go_offline()
%matplotlib inline

In [2]:
data= pd.read_csv('bitstampUSD_1-min_data_2012-01-01_to_2018-11-11.csv')

In [3]:
data.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3603136 entries, 0 to 3603135
Data columns (total 8 columns):
Timestamp            int64
Open                 float64
High                 float64
Low                  float64
Close                float64
Volume_(BTC)         float64
Volume_(Currency)    float64
Weighted_Price       float64
dtypes: float64(7), int64(1)
memory usage: 219.9 MB


In [5]:
data.describe()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
count,3.603136e+06,2.388829e+06,2.388829e+06,2.388829e+06,2.388829e+06,2.388829e+06,2.388829e+06,2.388829e+06
mean,1.433629e+09,2.571487e+03,2.573625e+03,2.569058e+03,2.571462e+03,1.092451e+01,2.369349e+04,2.571320e+03
std,6.257396e+07,3.629093e+03,3.632869e+03,3.624712e+03,3.629059e+03,3.560609e+01,8.921983e+04,3.628797e+03
min,1.325318e+09,3.800000e+00,3.800000e+00,1.500000e+00,1.500000e+00,0.000000e+00,0.000000e+00,3.800000e+00
25%,1.379365e+09,3.235200e+02,3.238000e+02,3.232900e+02,3.235100e+02,4.509416e-01,2.267963e+02,3.235000e+02
50%,1.433800e+09,6.227900e+02,6.230300e+02,6.223300e+02,6.228200e+02,2.092143e+00,1.595989e+03,6.227381e+02
75%,1.487847e+09,4.200930e+03,4.204930e+03,4.198480e+03,4.200880e+03,8.409572e+00,1.195215e+04,4.201000e+03
max,1.541894e+09,1.966576e+04,1.966600e+04,1.964996e+04,1.966575e+04,5.853852e+03,5.483271e+06,1.966330e+04


In [6]:
data.dropna(inplace=True)

In [7]:
data.reset_index(inplace=True)

In [8]:
data.drop('index',inplace=True,axis=1)

In [9]:
data.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
2,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
3,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
4,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000


In [10]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'],unit='s').dt.date

In [11]:
data.tail()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
2388824,2018-11-10,6348.54,6348.54,6348.54,6348.54,0.007997,50.769274,6348.540000
2388825,2018-11-10,6348.54,6348.54,6348.54,6348.54,0.007997,50.769274,6348.540000
2388826,2018-11-10,6348.54,6349.01,6348.54,6349.01,0.011729,74.466671,6348.936090
2388827,2018-11-10,6349.01,6349.01,6349.01,6349.01,0.068436,434.503642,6349.010000
2388828,2018-11-10,6349.17,6349.32,6349.17,6349.32,0.038261,242.927410,6349.214148


In [12]:
grouped = data.groupby('Timestamp')

In [13]:
real_price = grouped.mean()['Weighted_Price']

In [14]:
real_price.shape

(2504,)

In [15]:
real_price.iplot()

## DATA PREPROCESSING

In [16]:
dataset_train = real_price.iloc[:len(real_price)-365]
dataset_test = real_price.iloc[len(dataset_train):]

In [17]:
dataset_train = np.array(dataset_train)
dataset_train = dataset_train.reshape(dataset_train.shape[0],1)

In [18]:
from sklearn.preprocessing import MinMaxScaler

In [19]:
ms = MinMaxScaler(feature_range=(0,1))

In [20]:
dataset_train_scaled = ms.fit_transform(dataset_train)

In [21]:
dataset_train_scaled.shape

(2139, 1)

## CREATE A DATA STRUCTURE OF TIMESTEPS 5 TO 1 OUTPUT

In [22]:
from tqdm import tqdm

In [23]:
X_train = []
y_train = []

In [24]:
for i in tqdm(range(50,dataset_train_scaled.shape[0])):
    X_train.append(dataset_train_scaled[i-50:i,0])
    y_train.append(dataset_train_scaled[i,0])

100%|██████████████████████████████████████████████████████████████████████████| 2089/2089 [00:00<00:00, 113910.75it/s]


In [25]:
X_train, y_train = np.array(X_train),np.array(y_train)

In [26]:
def split_sequence(seq, n_steps_in, n_steps_out):
    """
    Splits the univariate time sequence
    """
    X, y = [], []
    
    for i in range(len(seq)):
        end = i + n_steps_in
        out_end = end + n_steps_out
        
        if out_end > len(seq):
            break
        
        seq_x, seq_y = seq[i:end], seq[end:out_end]
        
        X.append(seq_x)
        y.append(seq_y)
    
    return np.array(X), np.array(y)

In [27]:
#X_train,y_train = split_sequence(dataset_train_scaled,50,10)

### RESHAPE THE MODEL INCASE WE WANNA ADD MORE DIMENSIONS

In [28]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)

In [29]:
X_train.shape

(2089, 50, 1)

In [30]:
y_train.shape

(2089,)

In [31]:
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,Dropout,Bidirectional

In [32]:
model = keras.Sequential([
    LSTM(units=128,return_sequences=True,input_shape=(X_train.shape[1],1)),
    Dropout(0.2),
    LSTM(units=128,return_sequences=True),
    Dropout(0.2),
    LSTM(units=128,return_sequences=True),
    Dropout(0.2),
    LSTM(units=128,return_sequences=False),
    Dropout(0.2),
    Dense(1)#10 for 10 days prediction
])

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 128)           66560     
_________________________________________________________________
dropout (Dropout)            (None, 50, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 128)           131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 128)           131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               1

In [34]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae','mse']
             )

In [35]:
model.fit(X_train,y_train,epochs=20,verbose=0)

In [36]:
inputs = real_price[len(real_price)-len(dataset_test)-50:] #time step is 50 thats why we minus 50

In [37]:
"""nxt =model.predict(ms.transform(dataset_train[-50:]).reshape(1,50,1)).tolist()[0]
ms.inverse_transform(np.array(nxt).reshape(-1,1)).tolist()"""

'nxt =model.predict(ms.transform(dataset_train[-50:]).reshape(1,50,1)).tolist()[0]\nms.inverse_transform(np.array(nxt).reshape(-1,1)).tolist()'

In [38]:
'''real_life = dataset_train[len(dataset_train)-50:]
len(real_life)
real_life = ms.transform(np.array(real_life).reshape(-1,1))
real_test = []
for i in range(50,real_life.shape[0]+1):
    real_test.append(real_life[i-50:i,0])
real_test =np.array(real_test)
real_test = real_test.reshape(real_test.shape[0],real_test.shape[1],1)
ms.inverse_transform(model.predict(real_test))
'''

'real_life = dataset_train[len(dataset_train)-50:]\nlen(real_life)\nreal_life = ms.transform(np.array(real_life).reshape(-1,1))\nreal_test = []\nfor i in range(50,real_life.shape[0]+1):\n    real_test.append(real_life[i-50:i,0])\nreal_test =np.array(real_test)\nreal_test = real_test.reshape(real_test.shape[0],real_test.shape[1],1)\nms.inverse_transform(model.predict(real_test))\n'

In [39]:
inputs = inputs.values.reshape(-1,1)

In [40]:
inputs = ms.transform(inputs)

In [41]:
X_test =[]

In [42]:
for i in range(50,inputs.shape[0]):
    X_test.append(inputs[i-50:i,0])

In [43]:
X_test = np.array(X_test)

In [44]:
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [45]:
predicted_data = model.predict(X_test)

In [46]:
predicted_data = ms.inverse_transform(predicted_data)

In [47]:
y_test=np.array(dataset_test)
y_test=y_test.reshape(len(y_test),1)

In [48]:
#plt.figure(figsize=(16,8))
#sns.lineplot(x=dataset_test.index,y=pd.DataFrame(y_test)[0])
#sns.lineplot(x=dataset_test.index,y=pd.DataFrame(predicted_data)[0])
#plt.tight_layout()
myPLOT = pd.DataFrame(index=dataset_test.index,columns=['original','predicted'])
myPLOT['original'] = y_test[:,0]
myPLOT['predicted'] = predicted_data[:,0]

In [49]:
myPLOT.iplot()